<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# Alex's first tests towards hyperboloidal wave equation, based on:


# Start-to-Finish Example: Numerical Solution of the Scalar Wave Equation, in Curvilinear Coordinates

## Author: Zach Etienne
### Formatting improvements courtesy Brandon Clark

## This module solves the scalar wave equation in *spherical coordinates* (though other coordinates, including Cartesian, may be chosen).

**Notebook Status:** <font color ="green"><b> Validated </b></font>

**Validation Notes:** This module has been validated to converge at the expected order to the exact solution (see [plot](#convergence) at bottom).

### NRPy+ Source Code for this module: 
* [ScalarWave/ScalarWaveCurvilinear_RHSs.py](../edit/ScalarWave/ScalarWaveCurvilinear_RHSs.py) [\[**tutorial**\]](Tutorial-ScalarWaveCurvilinear.ipynb) Generates the right-hand side for the Scalar Wave Equation in curvilinear coordinates
* [ScalarWave/InitialData.py](../edit/ScalarWave/InitialData.py) [\[**tutorial**\]](Tutorial-ScalarWave.ipynb) Generating C code for either plane wave or spherical Gaussian initial data for the scalar wave equation 

## Introduction:
As outlined in the [previous NRPy+ tutorial notebook](Tutorial-ScalarWaveCurvilinear.ipynb), we first use NRPy+ to generate initial data for the scalar wave equation, and then we use it to generate the RHS expressions for [Method of Lines](https://reference.wolfram.com/language/tutorial/NDSolveMethodOfLines.html) time integration based on the [explicit Runge-Kutta fourth-order scheme](https://en.wikipedia.org/wiki/Runge%E2%80%93Kutta_methods) (RK4).

The entire algorithm is outlined below, with NRPy+-based components highlighted in <font color='green'>green</font>.

1. Allocate memory for gridfunctions, including temporary storage for the RK4 time integration.
1. <font color='green'>Set gridfunction values to initial data.</font>
1. Evolve the system forward in time using RK4 time integration. At each RK4 substep, do the following:
    1. <font color='green'>Evaluate scalar wave RHS expressions.</font>
    1. Apply boundary conditions.
1. At the end of each iteration in time, output the relative error between numerical and exact solutions.

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This notebook is organized as follows

1. [Step 1](#writec): Generate C code to solve the scalar wave equation in curvilinear coordinates
    1. [Step 1.a](#id_rhss): C code generation: Initial data and scalar wave right-hand-sides
    1. [Step 1.b](#boundaryconditions): C code generation: Boundary condition driver
    1. [Step 1.c](#cparams_rfm_and_domainsize): Generate Cparameters files; set reference metric parameters, including `domain_size`
    1. [Step 1.d](#cfl): C code generation: Finding the minimum proper distance between grid points, needed for [CFL](https://en.wikipedia.org/w/index.php?title=Courant%E2%80%93Friedrichs%E2%80%93Lewy_condition&oldid=806430673)-limited timestep
1. [Step 2](#mainc): `ScalarWaveCurvilinear_Playground.c`: The Main C Code
1. [Step 3](#compileexec): Compile generated C codes & solve the scalar wave equation
1. [Step 4](#convergence): Code validation: Plot the numerical error, and confirm that it converges to zero at expected rate with increasing numerical resolution (sampling)
1. [Step 5](#latex_pdf_output): Output this notebook to $\LaTeX$-formatted PDF file

<a id='writec'></a>

# Step 1: Using NRPy+ to generate necessary C code to solve the scalar wave equation in curvilinear, singular coordinates \[Back to [top](#toc)\]
$$\label{writec}$$

<a id='id_rhss'></a>

## Step 1.a: C code generation: Initial data and scalar wave RHSs \[Back to [top](#toc)\]
$$\label{id_rhss}$$


We choose simple plane wave initial data, which is documented in the [Cartesian scalar wave module](Tutorial-ScalarWave.ipynb). Specifically, we implement monochromatic (single-wavelength) wave traveling in the $\hat{k}$ direction with speed $c$
$$u(\vec{x},t) = f(\hat{k}\cdot\vec{x} - c t),$$
where $\hat{k}$ is a unit vector.

The scalar wave RHSs in curvilinear coordinates (documented [in the previous module](Tutorial-ScalarWaveCurvilinear.ipynb)) are simply the right-hand sides of the scalar wave equation written in curvilinear coordinates
\begin{align}
\partial_t u &= v \\
\partial_t v &= c^2 \left(\hat{g}^{ij} \partial_{i} \partial_{j} u - \hat{\Gamma}^i \partial_i u\right),
\end{align}
where $\hat{g}^{ij}$ is the inverse reference 3-metric (i.e., the metric corresponding to the underlying coordinate system we choose$-$spherical coordinates in our example below), and $\hat{\Gamma}^i$ is the contracted Christoffel symbol $\hat{\Gamma}^\tau = \hat{g}^{\mu\nu} \hat{\Gamma}^\tau_{\mu\nu}$.

Below we generate 
+ the initial data by calling `InitialData(Type="PlaneWave")` inside the NRPy+ [ScalarWave/InitialData.py](../edit/ScalarWave/InitialData.py) module (documented in [this NRPy+ Jupyter notebook](Tutorial-ScalarWave.ipynb)), and 
+ the RHS expressions by calling `ScalarWaveCurvilinear_RHSs()` inside the NRPy+ [ScalarWave/ScalarWaveCurvilinear_RHSs.py](../edit/ScalarWave/ScalarWaveCurvilinear_RHSs.py) module (documented in [this NRPy+ Jupyter notebook](Tutorial-ScalarWaveCurvilinear.ipynb)).

In [1]:
import sympy

In [2]:
# Step P1: Import needed NRPy+ core modules:
from outputC import lhrh,outCfunction # NRPy+: Core C code output module
import finite_difference as fin       # NRPy+: Finite difference C code generation module
import NRPy_param_funcs as par        # NRPy+: Parameter interface
import grid as gri                    # NRPy+: Functions having to do with numerical grids
import indexedexp as ixp              # NRPy+: Symbolic indexed expression (e.g., tensors, vectors, etc.) support
import reference_metric as rfm        # NRPy+: Reference metric support
import cmdline_helper as cmd          # NRPy+: Multi-platform Python command-line interface
import shutil, os, sys                # Standard Python modules for multiplatform OS-level functions

# Step P2: Create C code output directory:
Ccodesdir = os.path.join("ScalarWaveCurvilinear_Playground_Ccodes/")
# First remove C code output directory if it exists
# Courtesy https://stackoverflow.com/questions/303200/how-do-i-remove-delete-a-folder-that-is-not-empty
shutil.rmtree(Ccodesdir, ignore_errors=True)
# Then create a fresh directory
cmd.mkdir(Ccodesdir)

# Step P3: Create executable output directory:
#outdir = os.path.join(Ccodesdir,"output/")
outdir = os.path.join("output3Dhypwave/")
cmd.mkdir(outdir)

# Step 1: Set the spatial dimension parameter
#         to three this time, and then read
#         the parameter as DIM.
par.set_parval_from_str("grid::DIM",3)
DIM = par.parval_from_str("grid::DIM")

# Step 2: Set some core parameters, including CoordSystem, boundary condition,
#                                             MoL, timestepping algorithm, FD order,
#                                             floating point precision, and CFL factor:

# Step 2.a: Set the coordinate system for the numerical grid
# Choices are: Spherical, SinhSpherical, SinhSphericalv2, Cylindrical, SinhCylindrical,
#              SymTP, SinhSymTP
CoordSystem     = "Spherical" #"Cartesian" #"Spherical"
par.set_parval_from_str("reference_metric::CoordSystem",CoordSystem)
rfm.reference_metric()

# Step  2.b: Set boundary conditions
# Current choices are QuadraticExtrapolation (quadratic polynomial extrapolation) or Sommerfeld
BoundaryCondition = "QuadraticExtrapolation"

# Step 2.c: Set defaults for Coordinate system parameters.
#           These are perhaps the most commonly adjusted parameters,
#           so we enable modifications at this high level.

# domain_size sets the default value for:
#   * Spherical's params.RMAX
#   * SinhSpherical*'s params.AMAX
#   * Cartesians*'s -params.{x,y,z}min & .{x,y,z}max
#   * Cylindrical's -params.ZMIN & .{Z,RHO}MAX
#   * SinhCylindrical's params.AMPL{RHO,Z}
#   * *SymTP's params.AMAX
domain_size     = 1.0 # Needed for all coordinate systems.

# sinh_width sets the default value for:
#   * SinhSpherical's params.SINHW
#   * SinhCylindrical's params.SINHW{RHO,Z}
#   * SinhSymTP's params.SINHWAA
sinh_width      = 0.4 # If Sinh* coordinates chosen

# sinhv2_const_dr sets the default value for:
#   * SinhSphericalv2's params.const_dr
#   * SinhCylindricalv2's params.const_d{rho,z}
sinhv2_const_dr = 0.05# If Sinh*v2 coordinates chosen

# SymTP_bScale sets the default value for:
#   * SinhSymTP's params.bScale
SymTP_bScale    = 1.0 # If SymTP chosen

#default_KO_strength = 0.1

# Step 2.d: Set the order of spatial and temporal derivatives;
#           the core data type, and the CFL factor.
# RK_method choices include: Euler, "RK2 Heun", "RK2 MP", "RK2 Ralston", RK3, "RK3 Heun", "RK3 Ralston",
#              SSPRK3, RK4, DP5, DP5alt, CK5, DP6, L6, DP8
RK_method = "RK4"
FD_order  = 2        # Finite difference order: even numbers only, starting with 2. 12 is generally unstable
REAL      = "double" # Best to use double here.
CFL_FACTOR= 1.0

# Step 3: Generate Runge-Kutta-based (RK-based) timestepping code.
#         Each RK substep involves two function calls:
#      3.A: Evaluate RHSs (RHS_string)
#      3.B: Apply boundary conditions (post_RHS_string)
import MoLtimestepping.C_Code_Generation as MoL
from MoLtimestepping.RK_Butcher_Table_Dictionary import Butcher_dict
RK_order  = Butcher_dict[RK_method][1]
cmd.mkdir(os.path.join(Ccodesdir,"MoLtimestepping/"))

RHS_string = "rhs_eval(&rfmstruct, &params, RK_INPUT_GFS, RK_OUTPUT_GFS);"

if BoundaryCondition == "QuadraticExtrapolation":
    # Extrapolation BCs are applied to the evolved gridfunctions themselves after the MoL update
    post_RHS_string = "apply_bcs_curvilinear(&params, &bcstruct, NUM_EVOL_GFS, evol_gf_parity, RK_OUTPUT_GFS);"
elif BoundaryCondition == "Sommerfeld":
    # Sommerfeld BCs are applied to the gridfunction RHSs directly
    RHS_string += "apply_bcs_sommerfeld(&params, xx, &bcstruct, NUM_EVOL_GFS, evol_gf_parity, RK_INPUT_GFS, RK_OUTPUT_GFS);"
    post_RHS_string = ""
else:
    print("Invalid choice of boundary condition")
    sys.exit(1)

MoL.MoL_C_Code_Generation(RK_method, RHS_string = RHS_string, post_RHS_string = post_RHS_string,
                          outdir = os.path.join(Ccodesdir,"MoLtimestepping/"))

# Step 4: Import the ScalarWave.InitialData module.
#         This command only declares ScalarWave initial data
#         parameters and the InitialData() function.
import ScalarWave.InitialData_FT1S_GBUF as swid

# Step 5: Import ScalarWave_RHSs module.
#         This command only declares ScalarWave RHS parameters
#         and the ScalarWave_RHSs function (called later)
#import ScalarWave.ScalarWaveCurvilinear_RHSs_MetricComponents_FT1S_CharacteristicFields as swrhs
import ScalarWave.ScalarWaveCurvilinear_RHSs_MetricComponents_FT1S_GBUF as swrhs

# Step 6: Set the finite differencing order to FD_order (set above).
par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER",FD_order)

# Step 7: Call the InitialData() function to set up initial data.
#         Options include:
#    "PlaneWave": monochromatic (single frequency/wavelength) plane wave
#    "SphericalGaussian": spherically symmetric Gaussian, with default stdev=3
swid.InitialData(CoordSystem=CoordSystem,Type="SphericalGaussian",default_sigma=1/4,default_center=1/4)

# Step 8: Generate SymPy symbolic expressions for
#         uu_rhs and vv_rhs; the ScalarWave RHSs.
#         This function also declares the uu and vv
#         gridfunctions, which need to be declared
#         to output even the initial data to C file.
cmd.mkdir(os.path.join(Ccodesdir,"rfm_files/"))
par.set_parval_from_str("reference_metric::enable_rfm_precompute","True")
par.set_parval_from_str("reference_metric::rfm_precompute_Ccode_outdir",os.path.join(Ccodesdir,"rfm_files/"))
swrhs.ScalarWaveCurvilinear_RHSs_MetricComponents()
# Step 8.a: Now that we are finished with all the rfm hatted
#           quantities, let's restore them to their closed-
#           form expressions.
par.set_parval_from_str("reference_metric::enable_rfm_precompute","False") # Reset to False to disable rfm_precompute.
rfm.ref_metric__hatted_quantities()

# Step 9: Copy SIMD/SIMD_intrinsics.h to $Ccodesdir/SIMD/SIMD_intrinsics.h
cmd.mkdir(os.path.join(Ccodesdir,"SIMD"))
shutil.copy(os.path.join("SIMD/")+"SIMD_intrinsics.h",os.path.join(Ccodesdir,"SIMD/"))

# Step 10: Generate all needed C functions
enable_FD_functions = False
par.set_parval_from_str("finite_difference::enable_FD_functions",enable_FD_functions)





desc="Part P3: Declare the function for the exact solution at a single point. time==0 corresponds to the initial data."
name="exact_solution_single_point"
outCfunction(
    outfile  = os.path.join(Ccodesdir,name+".h"), desc=desc, name=name,
    params   ="""const REAL xx0,const REAL xx1,const REAL xx2,const paramstruct *restrict params,
                        REAL *gg_exact,REAL *gp_exact, REAL *bb_exact,REAL *bp_exact, REAL *uu_exact,REAL *up_exact, REAL *fgauge_exact,REAL *fgaugep_exact,
                        REAL *chi_exact, REAL *chip_exact,REAL *chipp_exact, REAL *rcoord_exact, REAL *Radius_exact, REAL *Rprime_exact, REAL *sinth_exact, REAL *costh_exact,
                        REAL *psigD0_exact, REAL *psigD1_exact,REAL *psigD2_exact, REAL *psibD0_exact, REAL *psibD1_exact,REAL *psibD2_exact,
                        REAL *psiuD0_exact, REAL *psiuD1_exact,REAL *psiuD2_exact, REAL *psifD0_exact, REAL *psifD1_exact, REAL *psifD2_exact""",
    body     = fin.FD_outputC("returnstring",[lhrh(lhs="*gg_exact",rhs=swid.gg_ID),
                                              lhrh(lhs="*gp_exact",rhs=swid.gp_ID),
                                              lhrh(lhs="*bb_exact",rhs=swid.bb_ID),
                                              lhrh(lhs="*bp_exact",rhs=swid.bp_ID),
                                              lhrh(lhs="*uu_exact",rhs=swid.uu_ID),
                                              lhrh(lhs="*up_exact",rhs=swid.up_ID),
                                              lhrh(lhs="*fgauge_exact",rhs=swid.fgauge_ID),
                                              lhrh(lhs="*fgaugep_exact",rhs=swid.fgaugep_ID),
                                              lhrh(lhs="*chi_exact",rhs=swid.chi_ID),
                                              lhrh(lhs="*chip_exact",rhs=swid.chip_ID),
                                              lhrh(lhs="*chipp_exact",rhs=swid.chipp_ID),
                                              lhrh(lhs="*rcoord_exact",rhs=swid.rcoord_ID),
                                              lhrh(lhs="*Radius_exact",rhs=swid.Radius_ID),
                                              lhrh(lhs="*Rprime_exact",rhs=swid.Rprime_ID),
                                              lhrh(lhs="*sinth_exact",rhs=swid.sinth_ID),
                                              lhrh(lhs="*costh_exact",rhs=swid.costh_ID),
                                              lhrh(lhs="*psigD0_exact",rhs=swid.psig_ID[0]),
                                              lhrh(lhs="*psigD1_exact",rhs=swid.psig_ID[1]),
                                              lhrh(lhs="*psigD2_exact",rhs=swid.psig_ID[2]),
                                              lhrh(lhs="*psibD0_exact",rhs=swid.psib_ID[0]),
                                              lhrh(lhs="*psibD1_exact",rhs=swid.psib_ID[1]),
                                              lhrh(lhs="*psibD2_exact",rhs=swid.psib_ID[2]),
                                              lhrh(lhs="*psiuD0_exact",rhs=swid.psiu_ID[0]),
                                              lhrh(lhs="*psiuD1_exact",rhs=swid.psiu_ID[1]),
                                              lhrh(lhs="*psiuD2_exact",rhs=swid.psiu_ID[2]),
                                              lhrh(lhs="*psifD0_exact",rhs=swid.psif_ID[0]),
                                              lhrh(lhs="*psifD1_exact",rhs=swid.psif_ID[1]),
                                              lhrh(lhs="*psifD2_exact",rhs=swid.psif_ID[2])]),
    loopopts = "")






desc="Part P4: Declare the function for the exact solution at all points. time==0 corresponds to the initial data."
name="exact_solution_all_points"
outCfunction(
    outfile  = os.path.join(Ccodesdir,name+".h"), desc=desc, name=name,
    params   ="const paramstruct *restrict params,REAL *restrict xx[3], REAL *restrict in_gfs",
    body     ="""exact_solution_single_point(xx[0][i0],xx[1][i1],xx[2][i2],params,
                            &in_gfs[IDX4S(GGGF,i0,i1,i2)],
                            &in_gfs[IDX4S(GPGF,i0,i1,i2)],
                            &in_gfs[IDX4S(BBGF,i0,i1,i2)],
                            &in_gfs[IDX4S(BPGF,i0,i1,i2)],
                            &in_gfs[IDX4S(UUGF,i0,i1,i2)],
                            &in_gfs[IDX4S(UPGF,i0,i1,i2)],
                            &in_gfs[IDX4S(FGAUGEGF,i0,i1,i2)],
                            &in_gfs[IDX4S(FGAUGEPGF,i0,i1,i2)],
                            &in_gfs[IDX4S(CHIGF,i0,i1,i2)],
                            &in_gfs[IDX4S(CHIPGF,i0,i1,i2)],
                            &in_gfs[IDX4S(CHIPPGF,i0,i1,i2)],
                            &in_gfs[IDX4S(RCOORDGF,i0,i1,i2)],
                            &in_gfs[IDX4S(RADIUSGF,i0,i1,i2)],
                            &in_gfs[IDX4S(RPRIMEGF,i0,i1,i2)],
                            &in_gfs[IDX4S(SINTHGF,i0,i1,i2)],
                            &in_gfs[IDX4S(COSTHGF,i0,i1,i2)],
                            &in_gfs[IDX4S(PSIGD0GF,i0,i1,i2)],
                            &in_gfs[IDX4S(PSIGD1GF,i0,i1,i2)],
                            &in_gfs[IDX4S(PSIGD2GF,i0,i1,i2)],
                            &in_gfs[IDX4S(PSIBD0GF,i0,i1,i2)],
                            &in_gfs[IDX4S(PSIBD1GF,i0,i1,i2)],
                            &in_gfs[IDX4S(PSIBD2GF,i0,i1,i2)],
                            &in_gfs[IDX4S(PSIUD0GF,i0,i1,i2)],
                            &in_gfs[IDX4S(PSIUD1GF,i0,i1,i2)],
                            &in_gfs[IDX4S(PSIUD2GF,i0,i1,i2)],
                            &in_gfs[IDX4S(PSIFD0GF,i0,i1,i2)],
                            &in_gfs[IDX4S(PSIFD1GF,i0,i1,i2)],
                            &in_gfs[IDX4S(PSIFD2GF,i0,i1,i2)]);""",
    loopopts = "AllPoints")

desc="Part P5: Declare the function to evaluate the scalar wave RHSs"
includes = None
if enable_FD_functions:
    includes = ["finite_difference_functions.h"]
name="rhs_eval"
outCfunction(
    outfile  = os.path.join(Ccodesdir,name+".h"), includes=includes, desc=desc, name=name,
    params   ="""rfm_struct *restrict rfmstruct,const paramstruct *restrict params,
                 const REAL *restrict in_gfs, REAL *restrict rhs_gfs""",
    body     =fin.FD_outputC("returnstring",[lhrh(lhs=gri.gfaccess("rhs_gfs","gg"),rhs=swrhs.gg_rhs),
                                             lhrh(lhs=gri.gfaccess("rhs_gfs","gp"),rhs=swrhs.gp_rhs),
                                             lhrh(lhs=gri.gfaccess("rhs_gfs","bb"),rhs=swrhs.bb_rhs),
                                             lhrh(lhs=gri.gfaccess("rhs_gfs","bp"),rhs=swrhs.bp_rhs),
                                             lhrh(lhs=gri.gfaccess("rhs_gfs","uu"),rhs=swrhs.uu_rhs),
                                             lhrh(lhs=gri.gfaccess("rhs_gfs","up"),rhs=swrhs.up_rhs),
                                             lhrh(lhs=gri.gfaccess("rhs_gfs","fgauge"),rhs=swrhs.fgauge_rhs),
                                             lhrh(lhs=gri.gfaccess("rhs_gfs","fgaugep"),rhs=swrhs.fgaugep_rhs),
                                             lhrh(lhs=gri.gfaccess("rhs_gfs","chi"),rhs=swrhs.chi_rhs),
                                             lhrh(lhs=gri.gfaccess("rhs_gfs","chip"),rhs=swrhs.chip_rhs),
                                             lhrh(lhs=gri.gfaccess("rhs_gfs","chipp"),rhs=swrhs.chipp_rhs),
                                             lhrh(lhs=gri.gfaccess("rhs_gfs","rcoord"),rhs=swrhs.rcoord_rhs),
                                             lhrh(lhs=gri.gfaccess("rhs_gfs","Radius"),rhs=swrhs.Radius_rhs),
                                             lhrh(lhs=gri.gfaccess("rhs_gfs","Rprime"),rhs=swrhs.Rprime_rhs),
                                             lhrh(lhs=gri.gfaccess("rhs_gfs","sinth"),rhs=swrhs.sinth_rhs),
                                             lhrh(lhs=gri.gfaccess("rhs_gfs","costh"),rhs=swrhs.costh_rhs),
                                             lhrh(lhs=gri.gfaccess("rhs_gfs","psigD0"),rhs=swrhs.psig_rhsD[0]),
                                             lhrh(lhs=gri.gfaccess("rhs_gfs","psigD1"),rhs=swrhs.psig_rhsD[1]),
                                             lhrh(lhs=gri.gfaccess("rhs_gfs","psigD2"),rhs=swrhs.psig_rhsD[2]),
                                             lhrh(lhs=gri.gfaccess("rhs_gfs","psibD0"),rhs=swrhs.psib_rhsD[0]),
                                             lhrh(lhs=gri.gfaccess("rhs_gfs","psibD1"),rhs=swrhs.psib_rhsD[1]),
                                             lhrh(lhs=gri.gfaccess("rhs_gfs","psibD2"),rhs=swrhs.psib_rhsD[2]),
                                             lhrh(lhs=gri.gfaccess("rhs_gfs","psiuD0"),rhs=swrhs.psiu_rhsD[0]),
                                             lhrh(lhs=gri.gfaccess("rhs_gfs","psiuD1"),rhs=swrhs.psiu_rhsD[1]),
                                             lhrh(lhs=gri.gfaccess("rhs_gfs","psiuD2"),rhs=swrhs.psiu_rhsD[2]),
                                             lhrh(lhs=gri.gfaccess("rhs_gfs","psifD0"),rhs=swrhs.psif_rhsD[0]),
                                             lhrh(lhs=gri.gfaccess("rhs_gfs","psifD1"),rhs=swrhs.psif_rhsD[1]),
                                             lhrh(lhs=gri.gfaccess("rhs_gfs","psifD2"),rhs=swrhs.psif_rhsD[2])],
                             params="enable_SIMD=True"),
    loopopts = "InteriorPoints,enable_SIMD,enable_rfm_precompute")


import ScalarWave.Constraint_GBUF as con
con.Constraint()

#def constraint():
#start = time.time()
print("Generating optimized C code for constraint. May take a while, depending on CoordSystem.")
# Set up the C function for the constraint RHS
desc="Evaluate the constraint"
name="constraint"
outCfunction(
    outfile  = os.path.join(Ccodesdir,name+".h"), includes=includes, desc=desc, name=name,
    params   = """rfm_struct *restrict rfmstruct, const paramstruct *restrict params,
                  REAL *restrict in_gfs, REAL *restrict aux_gfs""",
    body     = fin.FD_outputC("returnstring",
                              [lhrh(lhs=gri.gfaccess("aux_gfs","CDG0"),rhs=con.CDg[0]),
                               lhrh(lhs=gri.gfaccess("aux_gfs","CDG1"),rhs=con.CDg[1]),
                               lhrh(lhs=gri.gfaccess("aux_gfs","CDG2"),rhs=con.CDg[2]),
                               lhrh(lhs=gri.gfaccess("aux_gfs","CDB0"),rhs=con.CDb[0]),
                               lhrh(lhs=gri.gfaccess("aux_gfs","CDB1"),rhs=con.CDb[1]),
                               lhrh(lhs=gri.gfaccess("aux_gfs","CDB2"),rhs=con.CDb[2]),
                               lhrh(lhs=gri.gfaccess("aux_gfs","CDU0"),rhs=con.CDu[0]),
                               lhrh(lhs=gri.gfaccess("aux_gfs","CDU1"),rhs=con.CDu[1]),
                               lhrh(lhs=gri.gfaccess("aux_gfs","CDU2"),rhs=con.CDu[2]),
                               lhrh(lhs=gri.gfaccess("aux_gfs","CDF0"),rhs=con.CDf[0]),
                               lhrh(lhs=gri.gfaccess("aux_gfs","CDF1"),rhs=con.CDf[1]),
                               lhrh(lhs=gri.gfaccess("aux_gfs","CDF2"),rhs=con.CDf[2])],
                               params="outCverbose=False"),
    loopopts = "InteriorPoints,enable_rfm_precompute")

#TOOK AWAY  FROM PARAMS
#end = time.time()
#print("(BENCH) Finished connstr C codegen in " + str(end - start) + " seconds.")
#return pickled_outC_function_dict(outC_function_dict)


# Step 10.b Output functions for computing all finite-difference stencils
if enable_FD_functions:
    fin.output_finite_difference_functions_h(path=Ccodesdir)


print(rfm.ReU[2]) #CHECK IF XX0 IS R OR r ... I THINK ITS r (THIS GOES IN THE DISSIPATION OPERATOR)

Output C function exact_solution_single_point() to file ScalarWaveCurvilinear_Playground_Ccodes/exact_solution_single_point.h
Output C function exact_solution_all_points() to file ScalarWaveCurvilinear_Playground_Ccodes/exact_solution_all_points.h
Output C function rhs_eval() to file ScalarWaveCurvilinear_Playground_Ccodes/rhs_eval.h
Generating optimized C code for constraint. May take a while, depending on CoordSystem.
Output C function constraint() to file ScalarWaveCurvilinear_Playground_Ccodes/constraint.h
1/(xx0*sin(xx1))


<a id='boundaryconditions'></a>

## Step 1.b: Output needed C code for boundary condition driver \[Back to [top](#toc)\]
$$\label{boundaryconditions}$$

In [3]:
import CurviBoundaryConditions.CurviBoundaryConditions_GBUF as cbcs
cbcs.Set_up_CurviBoundaryConditions(os.path.join(Ccodesdir,"boundary_conditions/"),
                                    Cparamspath=os.path.join("../"),
                                    BoundaryCondition=BoundaryCondition)
if BoundaryCondition == "Sommerfeld":
    bcs = cbcs.sommerfeld_boundary_condition_class(fd_order=2,
                                                 vars_radial_falloff_power_default=3,
                                                 vars_speed_default=1.,
                                                 vars_at_inf_default=0.)
    # bcs.vars_radpower.items()
    bcs.write_sommerfeld_file(Ccodesdir)

Wrote to file "ScalarWaveCurvilinear_Playground_Ccodes/boundary_conditions/parity_conditions_symbolic_dot_products.h"
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
Evolved parity: ( Radius:0, Rprime:0, bb:0, bp:0, chi:0, chip:0, chipp:0,
    costh:0, fgauge:0, fgaugep:0, gg:0, gp:0, psibD0:1, psibD1:2, psibD2:3,
    psifD0:1, psifD1:2, psifD2:3, psigD0:1, psigD1:2, psigD2:3, psiuD0:1,
    psiuD1:2, psiuD2:3, rcoord:0, sinth:0, up:0, uu:0 )
Auxiliary parity: ( CDB0:1, CDB1:2, CDB2:3, CDF0:1, CDF1:2, CDF2:3, CDG0:1,
    CDG1:2, CDG2:3, CDU0:1, CDU1:2, CDU2:3 )

Wrote to file "ScalarWaveCurvilinear_Playground_Ccodes/boundary_conditions/EigenCoord_Cart_to_xx.h"


<a id='cparams_rfm_and_domainsize'></a>

## Step 1.c: Output C codes needed for declaring and setting Cparameters; also set `free_parameters.h` \[Back to [top](#toc)\]
$$\label{cparams_rfm_and_domainsize}$$

Based on declared NRPy+ Cparameters, first we generate `declare_Cparameters_struct.h`, `set_Cparameters_default.h`, and `set_Cparameters[-SIMD].h`.

Then we output `free_parameters.h`, which sets initial data parameters, as well as grid domain & reference metric parameters, applying `domain_size` and `sinh_width`/`SymTP_bScale` (if applicable) as set above

In [4]:
# Step 1.c.i: Set free_parameters.h
with open(os.path.join(Ccodesdir,"free_parameters.h"),"w") as file:
    file.write("""
// Set free-parameter values.
params.time = 0.0; // Initial simulation time time corresponds to exact solution at time=0.
params.wavespeed = 1.0;\n""")

# Append to $Ccodesdir/free_parameters.h reference metric parameters based on generic
#    domain_size,sinh_width,sinhv2_const_dr,SymTP_bScale,
#    parameters set above.
rfm.out_default_free_parameters_for_rfm(os.path.join(Ccodesdir,"free_parameters.h"),
                                        domain_size,sinh_width,sinhv2_const_dr,SymTP_bScale)

# Step 1.c.ii: Generate set_Nxx_dxx_invdx_params__and__xx.h:
rfm.set_Nxx_dxx_invdx_params__and__xx_h(os.path.join(Ccodesdir))

# Step 1.c.iii: Generate xx_to_Cart.h, which contains xx_to_Cart() for
#               (the mapping from xx->Cartesian) for the chosen
#               CoordSystem:
rfm.xx_to_Cart_h("xx_to_Cart","./set_Cparameters.h",os.path.join(Ccodesdir,"xx_to_Cart.h"))

# Step 1.c.iv: Generate declare_Cparameters_struct.h, set_Cparameters_default.h, and set_Cparameters[].h
par.generate_Cparameters_Ccodes(os.path.join(Ccodesdir))

<a id='cfl'></a>

## Step 1.d: Output needed C code for finding the minimum proper distance between grid points, needed for [CFL](https://en.wikipedia.org/w/index.php?title=Courant%E2%80%93Friedrichs%E2%80%93Lewy_condition&oldid=806430673)-limited timestep \[Back to [top](#toc)\]
$$\label{cfl}$$

In order for our explicit-timestepping numerical solution to the scalar wave equation to be stable, it must satisfy the [CFL](https://en.wikipedia.org/w/index.php?title=Courant%E2%80%93Friedrichs%E2%80%93Lewy_condition&oldid=806430673) condition:
$$
\Delta t \le \frac{\min(ds_i)}{c},
$$
where $c$ is the wavespeed, and
$$ds_i = h_i \Delta x^i$$ 
is the proper distance between neighboring gridpoints in the $i$th direction (in 3D, there are 3 directions), $h_i$ is the $i$th reference metric scale factor, and $\Delta x^i$ is the uniform grid spacing in the $i$th direction:

In [5]:
# Output the find_timestep() function to a C file.
rfm.out_timestep_func_to_file(os.path.join(Ccodesdir,"find_timestep.h"))

<a id='mainc'></a>

# Step 2: `ScalarWaveCurvilinear_Playground.c`: The Main C Code \[Back to [top](#toc)\]
$$\label{mainc}$$

Just as in [the start-to-finish, solving the scalar wave equation in Cartesian coordinates module](Tutorial-Start_to_Finish-ScalarWave.ipynb), we will implement the scalar wave equation via the Method of Lines. As discussed above, the critical differences between this code and the Cartesian version are as follows:
1. The CFL-constrained timestep depends on the proper distance between neighboring gridpoints
1. The boundary conditions must account for the fact that ghost zone points lying in the domain exterior can map either to the interior of the domain, or lie on the outer boundary. In the former case, we simply copy the data from the interior. In the latter case, we apply the usual outer boundary conditions.
1. The numerical grids must be staggered to avoid direct evaluation of the equations on coordinate singularities.

In [6]:
# Part P0: Define REAL, set the number of ghost cells NGHOSTS (from NRPy+'s FD_CENTDERIVS_ORDER),
#          and set the CFL_FACTOR (which can be overwritten at the command line)

with open(os.path.join(Ccodesdir,"ScalarWaveCurvilinear_Playground_REAL__NGHOSTS__CFL_FACTOR.h"), "w") as file:
    file.write("""
// Part P0.a: Set the number of ghost cells, from NRPy+'s FD_CENTDERIVS_ORDER
#define NGHOSTS """+str(int(FD_order/2)+1)+"""
// Part P0.b: Set the numerical precision (REAL) to double, ensuring all floating point
//            numbers are stored to at least ~16 significant digits
#define REAL """+REAL+"""
// Part P0.c: Set the number of ghost cells, from NRPy+'s FD_CENTDERIVS_ORDER
REAL CFL_FACTOR = """+str(CFL_FACTOR)+";\n")

In [7]:
%%writefile $Ccodesdir/ScalarWaveCurvilinear_Playground_FT1S_GBUF.c

// Step P0: Define REAL and NGHOSTS; and declare CFL_FACTOR. This header is generated in NRPy+.
#include "ScalarWaveCurvilinear_Playground_REAL__NGHOSTS__CFL_FACTOR.h"

#include "rfm_files/rfm_struct__declare.h"

#include "declare_Cparameters_struct.h"

// All SIMD intrinsics used in SIMD-enabled C code loops are defined here:
#include "SIMD/SIMD_intrinsics.h"

// Step P1: Import needed header files
#include "stdio.h"
#include "stdlib.h"
#include "math.h"
#include "stdint.h" // Needed for Windows GCC 6.x compatibility
#ifndef M_PI
#define M_PI 3.141592653589793238462643383279502884L
#endif
#ifndef M_SQRT1_2
#define M_SQRT1_2 0.707106781186547524400844362104849039L
#endif

// Step P2: Declare the IDX4S(gf,i,j,k) macro, which enables us to store 4-dimensions of
//           data in a 1D array. In this case, consecutive values of "i"
//           (all other indices held to a fixed value) are consecutive in memory, where
//           consecutive values of "j" (fixing all other indices) are separated by
//           Nxx_plus_2NGHOSTS0 elements in memory. Similarly, consecutive values of
//           "k" are separated by Nxx_plus_2NGHOSTS0*Nxx_plus_2NGHOSTS1 in memory, etc.
#define IDX4S(g,i,j,k) \
( (i) + Nxx_plus_2NGHOSTS0 * ( (j) + Nxx_plus_2NGHOSTS1 * ( (k) + Nxx_plus_2NGHOSTS2 * (g) ) ) )
#define IDX3S(i,j,k) ( (i) + Nxx_plus_2NGHOSTS0 * ( (j) + Nxx_plus_2NGHOSTS1 * ( (k) ) ) )
#define LOOP_REGION(i0min,i0max, i1min,i1max, i2min,i2max) \
  for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++)
#define LOOP_ALL_GFS_GPS(ii) _Pragma("omp parallel for") \
  for(int (ii)=0;(ii)<Nxx_plus_2NGHOSTS_tot*NUM_EVOL_GFS;(ii)++)

// Step P3: Set UUGF and VVGF macros, as well as xx_to_Cart()
#include "boundary_conditions/gridfunction_defines.h"

// Step P4: Set xx_to_Cart(const paramstruct *restrict params,
//                     REAL *restrict xx[3],
//                     const int i0,const int i1,const int i2,
//                     REAL xCart[3]),
//           which maps xx->Cartesian via
//    {xx[0][i0],xx[1][i1],xx[2][i2]}->{xCart[0],xCart[1],xCart[2]}
#include "xx_to_Cart.h"

// Step P5: Defines set_Nxx_dxx_invdx_params__and__xx(const int EigenCoord, const int Nxx[3],
//                                       paramstruct *restrict params, REAL *restrict xx[3]),
//          which sets params Nxx,Nxx_plus_2NGHOSTS,dxx,invdx, and xx[] for
//          the chosen Eigen-CoordSystem if EigenCoord==1, or
//          CoordSystem if EigenCoord==0.
#include "set_Nxx_dxx_invdx_params__and__xx.h"

// Step P6: Include basic functions needed to impose curvilinear
//          parity and boundary conditions.
#include "boundary_conditions/CurviBC_include_Cfunctions.h"

// Step P7: Find the CFL-constrained timestep
#include "find_timestep.h"

// Part P8: Declare the function for the exact solution at a single point. time==0 corresponds to the initial data.
#include "exact_solution_single_point.h"

// Part P9: Declare the function for the exact solution at all points. time==0 corresponds to the initial data.
#include "exact_solution_all_points.h"

// Part P10: Declare the function to evaluate the scalar wave RHSs
#include "rhs_eval.h"

// Part P11: Declare the function to evaluate the constrants.... NOT SURE IF THIS GOES HERE
#include "constraint.h"

// main() function:
// Step 0: Read command-line input, set up grid structure, allocate memory for gridfunctions, set up coordinates
// Step 1: Set up scalar wave initial data
// Step 2: Output relative error between numerical and exact solution.
// Step 3: Evolve scalar wave initial data forward in time using Method of Lines with chosen RK-like algorithm,
//         applying quadratic extrapolation outer boundary conditions.
// Step 4: Free all allocated memory
int main(int argc, const char *argv[]) {
    paramstruct params;
#include "set_Cparameters_default.h"

    // Step 0a: Read command-line input, error out if nonconformant
    if(argc != 5 || atoi(argv[1]) < NGHOSTS || atoi(argv[2]) < NGHOSTS || atoi(argv[3]) < NGHOSTS) {
        printf("Error: Expected one command-line argument: ./ScalarWaveCurvilinear_Playground Nx0 Nx1 Nx2 runtag,\n");
        printf("where Nx[0,1,2] is the number of grid points in the 0, 1, and 2 directions.\n");
        printf("Nx[] MUST BE larger than NGHOSTS (= %d) and runtag must be a string.\n",NGHOSTS);
        exit(1);
    }
    // Step 0b: Set up numerical grid structure, first in space...
    const int Nxx[3] = { atoi(argv[1]), atoi(argv[2]), atoi(argv[3]) };
    if(Nxx[0]%2 != 0 || Nxx[1]%2 != 0 || Nxx[2]%2 != 0) {
        printf("Error: Cannot guarantee a proper cell-centered grid if number of grid cells not set to even number.\n");
        printf("       For example, in case of angular directions, proper symmetry zones will not exist.\n");
        exit(1);
    }

    // Step 0c: Set free parameters, overwriting Cparameters defaults
    //          by hand or with command-line input, as desired.
#include "free_parameters.h"

   // Step 0d: Uniform coordinate grids are stored to *xx[3]
    REAL *xx[3];
    // Step 0d.i: Set bcstruct
    bc_struct bcstruct;
    {
        int EigenCoord = 1;
        // Step 0d.ii: Call set_Nxx_dxx_invdx_params__and__xx(), which sets
        //             params Nxx,Nxx_plus_2NGHOSTS,dxx,invdx, and xx[] for the
        //             chosen Eigen-CoordSystem.
        set_Nxx_dxx_invdx_params__and__xx(EigenCoord, Nxx, &params, xx);
        // Step 0d.iii: Set Nxx_plus_2NGHOSTS_tot
#include "set_Cparameters-nopointer.h"
        const int Nxx_plus_2NGHOSTS_tot = Nxx_plus_2NGHOSTS0*Nxx_plus_2NGHOSTS1*Nxx_plus_2NGHOSTS2;
        // Step 0e: Find ghostzone mappings; set up bcstruct
#include "boundary_conditions/driver_bcstruct.h"
        // Step 0e.i: Free allocated space for xx[][] array
        for(int i=0;i<3;i++) free(xx[i]);
    }

    // Step 0f: Call set_Nxx_dxx_invdx_params__and__xx(), which sets
    //          params Nxx,Nxx_plus_2NGHOSTS,dxx,invdx, and xx[] for the
    //          chosen (non-Eigen) CoordSystem.
    int EigenCoord = 0;
    set_Nxx_dxx_invdx_params__and__xx(EigenCoord, Nxx, &params, xx);

    // Step 0g: Set all C parameters "blah" for params.blah, including
    //          Nxx_plus_2NGHOSTS0 = params.Nxx_plus_2NGHOSTS0, etc.
#include "set_Cparameters-nopointer.h"
    const int Nxx_plus_2NGHOSTS_tot = Nxx_plus_2NGHOSTS0*Nxx_plus_2NGHOSTS1*Nxx_plus_2NGHOSTS2;

    // Step 0h: Time coordinate parameters
    const REAL t_final = 10.0*domain_size; //2*domain_size;

    // Step 0i: Set timestep based on smallest proper distance between gridpoints and CFL factor
    REAL dt = 0.05/Nxx[0]/Nxx[1]; //Works unless suuper high resolution is used
    //REAL dt = 0.5*find_timestep(&params, xx); //# original
    printf("# Timestep set to = %e\n",(double)dt);
    int N_final = (int)(t_final / dt + 0.5); // The number of points in time.
                                             // Add 0.5 to account for C rounding down
                                             // typecasts to integers.
    int output_every_N = (int)((REAL)N_final/200.0);
    if(output_every_N == 0) output_every_N = 1;

    // Step 0j: Error out if the number of auxiliary gridfunctions outnumber evolved gridfunctions.
    //              This is a limitation of the RK method. You are always welcome to declare & allocate
    //              additional gridfunctions by hand.
    if(NUM_AUX_GFS > NUM_EVOL_GFS) {
        printf("Error: NUM_AUX_GFS > NUM_EVOL_GFS. Either reduce the number of auxiliary gridfunctions,\n");
        printf("       or allocate (malloc) by hand storage for *diagnostic_output_gfs. \n");
        exit(1);
    }

    // Step 0k: Allocate memory for gridfunctions
#include "MoLtimestepping/RK_Allocate_Memory.h"

    // Step 0l: Set up precomputed reference metric arrays
    // Step 0l.i: Allocate space for precomputed reference metric arrays.
#include "rfm_files/rfm_struct__malloc.h"

    // Step 0l.ii: Define precomputed reference metric arrays.
    {
#include "set_Cparameters-nopointer.h"
#include "rfm_files/rfm_struct__define.h"
    }

    // Step 1: Set up initial data to be exact solution at time=0:
    params.time = 0.0; exact_solution_all_points(&params, xx, y_n_gfs);
    apply_bcs_curvilinear(&params, &bcstruct, NUM_EVOL_GFS, evol_gf_parity, y_n_gfs);

    // Label tag of simulation:
    char tag[20];
    sprintf(tag,"%s",argv[4]);
    printf("Tag for output is: %s, final time is: %e.\n",argv[4],t_final);

    // Write grid to gridname, so it does not need to be included in the time evolution output.
    char gridname[100];
    sprintf(gridname,"grid%s%d.txt",tag,Nxx[0]);
    FILE *grid3D = fopen(gridname, "w");
    LOOP_REGION(NGHOSTS,Nxx_plus_2NGHOSTS0-NGHOSTS,
                NGHOSTS,Nxx_plus_2NGHOSTS1-NGHOSTS,
                NGHOSTS,Nxx_plus_2NGHOSTS2-NGHOSTS) {
        const int idx = IDX3S(i0,i1,i2);
        REAL xx0 = xx[0][i0];
        REAL xx1 = xx[1][i1];
        REAL xx2 = xx[2][i2];
        REAL xCart[3];
        xx_to_Cart(&params,xx,i0,i1,i2,xCart);
        fprintf(grid3D,"%e %e %e\n",
            //        xCart[0],xCart[1],xCart[2], // outputs Cartesian gridpoints
                xx0,xx1,xx2);
        }
        fclose(grid3D);

    for(int n=0;n<=N_final;n++)
       { // Main loop to progress forward in time.

        // Step 1a: Set current time to correct value & compute exact solution
        params.time = ((REAL)n)*dt;

        // Step 1b: Calculate the constraints ..... NOW I THINK THIS DOESNT GO HERE
        //#include "constraint.h"
        //constraint(&rfmstruct, &params, y_n_gfs, diagnostic_output_gfs);

        // Step 2: Code validation: Compute log of L2 norm of difference
        //         between numerical and exact solutions:
        //   log_L2_Norm = log10( sqrt[Integral( [numerical - exact]^2 * dV)] ),
        //         where integral is within 30% of the grid outer boundary (domain_size)
        if(n%output_every_N == 0) {
            printf("%e\n",(double)params.time);
            REAL integral = 0.0;
            REAL numpts   = 0.0;
            constraint(&rfmstruct, &params, y_n_gfs, diagnostic_output_gfs);
#pragma omp parallel for reduction(+:integral,numpts)
            LOOP_REGION(NGHOSTS,Nxx_plus_2NGHOSTS0-NGHOSTS,
                        NGHOSTS,Nxx_plus_2NGHOSTS1-NGHOSTS,
                        NGHOSTS,Nxx_plus_2NGHOSTS2-NGHOSTS) {
                REAL xCart[3]; xx_to_Cart(&params,xx,i0,i1,i2, xCart);
                if(sqrt(xCart[0]*xCart[0] + xCart[1]*xCart[1] + xCart[2]*xCart[2]) < domain_size*0.3) {
                    REAL gg_exact,gp_exact,
                    bb_exact,bp_exact,uu_exact,up_exact,fgauge_exact,fgaugep_exact,
                    chi_exact,chip_exact,chipp_exact,rcoord_exact,Radius_exact,Rprime_exact,sinth_exact,costh_exact,
                    psigD0_exact,psigD1_exact,psigD2_exact,psibD0_exact,psibD1_exact,psibD2_exact,
                    psiuD0_exact,psiuD1_exact,psiuD2_exact,psifD0_exact,psifD1_exact,psifD2_exact;
                    exact_solution_single_point(xx[0][i0],xx[1][i1],xx[2][i2],&params,
                    &gg_exact,&gp_exact,&bb_exact,&bp_exact, &uu_exact,&up_exact, &fgauge_exact,&fgaugep_exact,
                    &chi_exact,&chip_exact,&chipp_exact,&rcoord_exact,&Radius_exact,&Rprime_exact,&sinth_exact,&costh_exact,
                    &psigD0_exact,&psigD1_exact,&psigD2_exact,&psibD0_exact,&psibD1_exact,&psibD2_exact,
                    &psiuD0_exact,&psiuD1_exact,&psiuD2_exact,&psifD0_exact,&psifD1_exact,&psifD2_exact);
                    double num   = (double)y_n_gfs[IDX4S(GGGF,i0,i1,i2)];
                    double exact = (double)gg_exact;
                    integral += (num - exact)*(num - exact);
                    numpts   += 1.0;
                }
            }
            // Compute and output the log of the L2 norm.
//            REAL log_L2_Norm = log10(sqrt(integral/numpts));
//            printf("%e %e\n",(double)params.time,log_L2_Norm);

            // Write time evolution output of variables to files.
            char filename[100];
            sprintf(filename,"out%s%d-%08d.txt",tag,Nxx[0],n);
            FILE *out2D = fopen(filename, "w");
            fprintf(out2D,"%e\n",params.time);
            LOOP_REGION(NGHOSTS,Nxx_plus_2NGHOSTS0-NGHOSTS,
                        NGHOSTS,Nxx_plus_2NGHOSTS1-NGHOSTS,
                        NGHOSTS,Nxx_plus_2NGHOSTS2-NGHOSTS) {
                const int idx = IDX3S(i0,i1,i2);
                //REAL xx0 = xx[0][i0];
                //REAL xx1 = xx[1][i1];
                //REAL xx2 = xx[2][i2];
                //REAL xCart[3];
                //xx_to_Cart(&params,xx,i0,i1,i2,xCart);
                fprintf(out2D,"%e %e %e %e %e %e %e %e %e %e %e %e %e %e %e %e %e %e %e %e %e %e %e %e %e %e %e %e %e %e %e %e\n",
                //        xCart[0],xCart[1],xCart[2], // outputs Cartesian gridpoints
                        //xx0,xx1,xx2, // outputs gridpoints in original coordinates
                        y_n_gfs[IDX4S(GGGF,i0,i1,i2)],
                        y_n_gfs[IDX4S(GPGF,i0,i1,i2)],
                        y_n_gfs[IDX4S(PSIGD0GF,i0,i1,i2)],
                        y_n_gfs[IDX4S(PSIGD1GF,i0,i1,i2)],
                        y_n_gfs[IDX4S(PSIGD2GF,i0,i1,i2)],
                        y_n_gfs[IDX4S(BBGF,i0,i1,i2)],
                        y_n_gfs[IDX4S(BPGF,i0,i1,i2)],
                        y_n_gfs[IDX4S(PSIBD0GF,i0,i1,i2)],
                        y_n_gfs[IDX4S(PSIBD1GF,i0,i1,i2)],
                        y_n_gfs[IDX4S(PSIBD2GF,i0,i1,i2)],
                        y_n_gfs[IDX4S(UUGF,i0,i1,i2)],
                        y_n_gfs[IDX4S(UPGF,i0,i1,i2)],
                        y_n_gfs[IDX4S(PSIUD0GF,i0,i1,i2)],
                        y_n_gfs[IDX4S(PSIUD1GF,i0,i1,i2)],
                        y_n_gfs[IDX4S(PSIUD2GF,i0,i1,i2)],
                        y_n_gfs[IDX4S(FGAUGEGF,i0,i1,i2)],
                        y_n_gfs[IDX4S(FGAUGEPGF,i0,i1,i2)],
                        y_n_gfs[IDX4S(PSIFD0GF,i0,i1,i2)],
                        y_n_gfs[IDX4S(PSIFD1GF,i0,i1,i2)],
                        y_n_gfs[IDX4S(PSIFD2GF,i0,i1,i2)],
                        diagnostic_output_gfs[IDX4S(CDG0GF,i0,i1,i2)],
                        diagnostic_output_gfs[IDX4S(CDG1GF,i0,i1,i2)],
                        diagnostic_output_gfs[IDX4S(CDG2GF,i0,i1,i2)],
                        diagnostic_output_gfs[IDX4S(CDB0GF,i0,i1,i2)],
                        diagnostic_output_gfs[IDX4S(CDB1GF,i0,i1,i2)],
                        diagnostic_output_gfs[IDX4S(CDB2GF,i0,i1,i2)],
                        diagnostic_output_gfs[IDX4S(CDU0GF,i0,i1,i2)],
                        diagnostic_output_gfs[IDX4S(CDU1GF,i0,i1,i2)],
                        diagnostic_output_gfs[IDX4S(CDU2GF,i0,i1,i2)],
                        diagnostic_output_gfs[IDX4S(CDF0GF,i0,i1,i2)],
                        diagnostic_output_gfs[IDX4S(CDF1GF,i0,i1,i2)],
                        diagnostic_output_gfs[IDX4S(CDF2GF,i0,i1,i2)]);
            }
            fclose(out2D);

        }

        // Step 3: Step forward one timestep (t -> t+dt) in time using
        //           chosen RK-like MoL timestepping algorithm
#include "MoLtimestepping/RK_MoL.h"

    } // End main loop to progress forward in time.
    printf("%d\n",Nxx_plus_2NGHOSTS1);
    printf("%d\n",Nxx_plus_2NGHOSTS2);
    printf("%d\n",Nxx[1]+NGHOSTS);
    //printf("%d",bcstruct->inner[1][0].inner_bc_dest_pt.i0);
    printf("%d\n",NGHOSTS);
    // Step 4: Free all allocated memory
#include "rfm_files/rfm_struct__freemem.h"
#include "boundary_conditions/bcstruct_freemem.h"
#include "MoLtimestepping/RK_Free_Memory.h"
    for(int i=0;i<3;i++) free(xx[i]);
    return 0;
}

Writing ScalarWaveCurvilinear_Playground_Ccodes//ScalarWaveCurvilinear_Playground_FT1S_GBUF.c


<a id='compileexec'></a>

# Step 3: Compile generated C codes & solve the scalar wave equation \[Back to [top](#toc)\]
$$\label{compileexec}$$

To aid in the cross-platform-compatible (with Windows, MacOS, & Linux) compilation and execution, we make use of `cmdline_helper` [(**Tutorial**)](Tutorial-cmdline_helper.ipynb).

In [8]:
import cmdline_helper as cmd

cmd.C_compile(os.path.join(Ccodesdir,"ScalarWaveCurvilinear_Playground_FT1S_GBUF.c"),
              os.path.join(outdir,"ScalarWaveCurvilinear_Playground_FT1S_GBUF"),compile_mode="optimized")
# !clang -Ofast -fopenmp -mavx2 -mfma ScalarWave/ScalarWaveCurvilinear_Playground.c -o ScalarWaveCurvilinear_Playground -lm
# !icc -align -qopenmp -xHost -O2 -qopt-report=5 -qopt-report-phase ipo -qopt-report-phase vec -vec-threshold1 -qopt-prefetch=4 ScalarWave/ScalarWaveCurvilinear_Playground.c -o ScalarWaveCurvilinear_Playground
# !gcc-7 -Ofast -fopenmp -march=native ScalarWave/ScalarWaveCurvilinear_Playground.c -o ScalarWaveCurvilinear_Playground -lm

# Change to output directory
os.chdir(outdir)
# Clean up existing output files
cmd.delete_existing_files("outtestD12-*.txt")
# Run executable
if par.parval_from_str("reference_metric::CoordSystem") == "Cartesian":
    cmd.Execute("ScalarWaveCurvilinear_Playground", "16 16 16", "out-lowresolution.txt")
    #cmd.Execute("ScalarWaveCurvilinear_Playground", "24 24 24", "out-medresolution.txt")
    #cmd.Execute("ScalarWaveCurvilinear_Playground", "32 32 32 carttestsnodiss", "testarg")
    #cmd.Execute("ScalarWaveCurvilinear_Playground", "540 108 2", "out-higherresolution.txt")
else:
    #cmd.Execute("ScalarWaveCurvilinear_Playground", "16  8 16", "out-lowresolution.txt")
    #cmd.Execute("ScalarWaveCurvilinear_Playground", "24 12 24", "out-medresolution.txt")
    #cmd.Execute("ScalarWaveCurvilinear_Playground", "36 18 36", "out-highresolution.txt")
    #cmd.Execute("ScalarWaveCurvilinear_Playground", "32  8 16", "out-lowresolution.txt")
    #cmd.Execute("ScalarWaveCurvilinear_Playground", "48 12 24", "out-medresolution.txt")
    #cmd.Execute("ScalarWaveCurvilinear_Playground", "72 18 36", "out-highresolution.txt")
    #cmd.Execute("ScalarWaveCurvilinear_Playground", "100 2 2", "out-lowresolution.txt")
    #cmd.Execute("ScalarWaveCurvilinear_Playground", "200 2 2", "out-medresolution.txt")
    #cmd.Execute("ScalarWaveCurvilinear_Playground", "400 2 2", "out-highresolution.txt")
    #cmd.Execute("ScalarWaveCurvilinear_Playground", "800 2 2", "out-highresolution.txt")
    #cmd.Execute("ScalarWaveCurvilinear_Playground", "200 2 2", "out-lowresolution.txt")
    #cmd.Execute("ScalarWaveCurvilinear_Playground", "600 2 2", "out-medresolution.txt")
    #cmd.Execute("ScalarWaveCurvilinear_Playground", "1800 2 2", "out-highresolution.txt")
    #cmd.Execute("ScalarWaveCurvilinear_Playground", "200 8 16", "out-lowresolution.txt")
    #cmd.Execute("ScalarWaveCurvilinear_Playground", "300 12 24", "out-medresolution.txt")
    #cmd.Execute("ScalarWaveCurvilinear_Playground", "450 18 36", "out-highresolution.txt")
    #cmd.Execute("ScalarWaveCurvilinear_Playground", "160 8 24", "out-lowresolution.txt")
    #cmd.Execute("ScalarWaveCurvilinear_Playground", "240 12 36", "out-medresolution.txt")
    #cmd.Execute("ScalarWaveCurvilinear_Playground", "360 18 54", "out-highresolution.txt")
    #cmd.Execute("ScalarWaveCurvilinear_Playground", "160 8 32", "out-lowresolution.txt")
    #cmd.Execute("ScalarWaveCurvilinear_Playground", "240 12 48", "out-medresolution.txt")
    #cmd.Execute("ScalarWaveCurvilinear_Playground", "360 18 72", "out-highresolution.txt")
    #cmd.Execute("ScalarWaveCurvilinear_Playground", "160 16 24 testD", "out-lowresolution.txt")
    #cmd.Execute("ScalarWaveCurvilinear_Playground", "240 24 36", "out-medresolution.txt")
    #cmd.Execute("ScalarWaveCurvilinear_Playground", "360 36 54", "out-highresolution.txt")
    #cmd.Execute("ScalarWaveCurvilinear_Playground", "200 32 2", "out-lowresolution.txt")
    #cmd.Execute("ScalarWaveCurvilinear_Playground", "300 48 2", "out-medresolution.txt")
    #cmd.Execute("ScalarWaveCurvilinear_Playground", "450 72 2", "out-highresolution.txt")
#     cmd.Execute("ScalarWaveCurvilinear_Playground", "32 32 2", "out-lowresolution.txt")
#     cmd.Execute("ScalarWaveCurvilinear_Playground", "48 48 2", "out-medresolution.txt")
#     cmd.Execute("ScalarWaveCurvilinear_Playground", "72 72 2", "out-highresolution.txt")
#    cmd.Execute("ScalarWaveCurvilinear_Playground", "160 32 2", "out-lowresolution.txt")
#    cmd.Execute("ScalarWaveCurvilinear_Playground", "240 48 2", "out-medresolution.txt")
#    cmd.Execute("ScalarWaveCurvilinear_Playground", "360 72 2", "out-highresolution.txt")
#    cmd.Execute("ScalarWaveCurvilinear_Playground", "540 108 2", "out-higherresolution.txt")
#    cmd.Execute("ScalarWaveCurvilinear_Playground", "20 12 2", "out-lowresolution.txt")
#    cmd.Execute("ScalarWaveCurvilinear_Playground", "60 36 2", "out-medresolution.txt")
#    cmd.Execute("ScalarWaveCurvilinear_Playground", "180 108 2", "out-highresolution.txt")
#    cmd.Execute("ScalarWaveCurvilinear_Playground", "540 108 2", "out-higherresolution.txt")
    #cmd.Execute("ScalarWaveCurvilinear_Playground", "12 12 12 testD", "testarg")
    #cmd.Execute("ScalarWaveCurvilinear_Playground_FT1S", "20 10 12 testD", "testarg")
    #cmd.Execute("ScalarWaveCurvilinear_Playground_FT1S", "60 10 12 testD", "testarg")
    #cmd.Execute("ScalarWaveCurvilinear_Playground_FT1S", "180 10 12 testD", "testarg")
    #cmd.Execute("ScalarWaveCurvilinear_Playground_FT1S", "100 2 2 testDConstr", "testarg")
    #cmd.Execute("ScalarWaveCurvilinear_Playground_FT1S_GBUF", "80 2 2 GBUF_SphSymm", "testarg")
    cmd.Execute("ScalarWaveCurvilinear_Playground_FT1S_GBUF", "80 12 24 GBUF_noSymm", "testarg")
    #cmd.Execute("ScalarWaveCurvilinear_Playground_FT1S", "200 2 2 testDConstr", "testarg")
    #cmd.Execute("ScalarWaveCurvilinear_Playground_FT1S", "100 2 2 testDQuad", "testarg")
    #cmd.Execute("ScalarWaveCurvilinear_Playground_FT1S", "200 2 2 testDQuad", "testarg")
    #cmd.Execute("ScalarWaveCurvilinear_Playground_FT1S", "400 2 2 testDQuad", "testarg")
    #cmd.Execute("ScalarWaveCurvilinear_Playground_FT1S", "80 12 24 testD", "testarg")
    #cmd.Execute("ScalarWaveCurvilinear_Playground_FT1S", "80 12 24 testDConv", "testarg")
    ##cmd.Execute("ScalarWaveCurvilinear_Playground_FT1S", "80 12 48 testDConv2", "testarg")
    #cmd.Execute("ScalarWaveCurvilinear_Playground_FT1S_Hyperboloidal", "80 24 24 testHNS", "testarg")
    ##cmd.Execute("ScalarWaveCurvilinear_Playground_FT1S", "80 12 192 testDConv4", "testarg")
    ##cmd.Execute("ScalarWaveCurvilinear_Playground_FT1S", "80 12 384 testDConv5", "testarg")

# Return to root directory
os.chdir(os.path.join("../../"))

Compiling executable...
(EXEC): Executing `gcc -std=gnu99 -Ofast -fopenmp -march=native -funroll-loops ScalarWaveCurvilinear_Playground_Ccodes/ScalarWaveCurvilinear_Playground_FT1S_GBUF.c -o output3Dhypwave/ScalarWaveCurvilinear_Playground_FT1S_GBUF -lm`...
(BENCH): Finished executing in 3.2066588401794434 seconds.
Finished compilation.
(EXEC): Executing `taskset -c 0,1,2,3,4,5 ./ScalarWaveCurvilinear_Playground_FT1S_GBUF 80 12 24 GBUF_noSymm`...


KeyboardInterrupt: 